In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [ ]:
from skimage.io import imread,imshow

In [ ]:
from skimage.transform import resize
from sklearn.utils import shuffle
from tqdm import tqdm

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer,Conv2D,MaxPool2D,BatchNormalization,Dropout,Flatten,Dense
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
train_dataset_0_all=glob.glob('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/all/*.bmp')
train_dataset_0_hem=glob.glob('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/hem/*.bmp')
train_dataset_1_all=glob.glob('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1/all/*.bmp')
train_dataset_1_hem=glob.glob('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1/hem/*.bmp')
train_dataset_2_all=glob.glob('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_2/all/*.bmp')
train_dataset_2_hem=glob.glob('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_2/hem/*.bmp')



In [ ]:
valid_data=pd.read_csv('../input/leukemia-classification/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data_labels.csv')

In [ ]:
a,b,c=len(train_dataset_0_all),len(train_dataset_1_all),len(train_dataset_2_all)
d=a+b+c
print('count:',d)

In [ ]:
a,b,c=len(train_dataset_0_hem),len(train_dataset_1_hem),len(train_dataset_2_hem)
d=a+b+c
print('count:',d)

In [ ]:
valid_data.head()

In [ ]:
A=[]
H=[]
A.extend(train_dataset_0_all)
A.extend(train_dataset_1_all)
A.extend(train_dataset_2_all)
H.extend(train_dataset_0_hem)
H.extend(train_dataset_1_hem)
H.extend(train_dataset_2_hem)
print(len(A))
print(len(H))

In [ ]:
A=np.array(A)
H=np.array(H)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 10, figsize = (20,20))
for i in tqdm(range(0,10)):
              rand=np.random.randint(len(A))
              img=imread(A[rand])
              img=resize(img,(128,128))
              ax[i].imshow(img)

            
    

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 10, figsize = (20,20))
for i in tqdm(range(0,10)):
              rand=np.random.randint(len(H))
              img=imread(H[rand])
              img=resize(img,(128,128))
              ax[i].imshow(img)

In [ ]:
image=[]
label=[]
for i in tqdm(range(len(A))):
              img=imread(A[i])
              img=resize(img,(128,128))
              image.append(img)
              label.append(1)
              
for i in tqdm(range(len(H))):
              img_=imread(H[i])
              img=resize(img,(128,128))
              image.append(img)
              label.append(0)

image=np.array(image)
label=np.array(label)
              

In [ ]:
del A 
del H

In [ ]:
image, label = shuffle(image, label, random_state = 42)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize = (20,20))
for i in tqdm(range(0,5)):
              rand=np.random.randint(len(image))
              ax[i].imshow(image[rand])
              a=label[rand]
              if a==1:
                ax[i].set_title('diseased')
              else:
                ax[i].set_title('fine')
                

In [ ]:
X=image
y=label

In [ ]:
del image
del label


In [ ]:

X_val = []

for image_name in valid_data.new_names:
    # Loading images
    img = imread('../input/leukemia-classification/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data/' + image_name)
    # Resizing 
    img = resize(img, (128,128))
    # Appending them into list
    X_val.append(img)
 
# Converting into array
X_val = np.array(X_val)


# Storing target values as well 
y_val = valid_data.labels.values

In [ ]:
train_datagen  = ImageDataGenerator(horizontal_flip=True,
                                    vertical_flip=True,
                                    zoom_range = 0.2)

train_datagen.fit(X)

In [ ]:
model=Sequential()

In [ ]:
model.add(InputLayer(input_shape=(128,128,3)))

model.add(Conv2D(filters=32,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),padding='valid'))
model.add(Dropout(.2))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),padding='valid'))
model.add(Dropout(.2))

model.add(Conv2D(filters=128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),padding='valid'))
model.add(Dropout(.2))

model.add(Flatten())

model.add(Dense(units=1,activation='sigmoid'))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 15)

In [ ]:
len(X),len(X_val)

In [ ]:
model.fit(train_datagen.flow(X, y, batch_size = 512),
          validation_data = (X_val, y_val),
          epochs = 500,
          verbose = 1,
          callbacks =[earlystopping])


In [ ]:
model.summary

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()